In [1]:
# Soft Color Segmentation Prorotype

# The Algorithme is composed of three stages
# 1. Color Unmixing
# 2. Matte Regularisation
# 3. Color Refinement

import cv2
from cv2.ximgproc import guidedFilter
import numpy as np
from scipy.stats import multivariate_normal
from scipy.spatial.distance import mahalanobis
import matplotlib.pyplot as plt
import math
import sys
import threading
import concurrent.futures

# Returns Per Pixel Vote Results And Winner Bin
def voteOnBin(img, gradient, representation_score, representation_threshold):
    # Voting on the bins
    votes_per_bin = {}
    votes = np.zeros((img.shape[0], img.shape[1], 1))
    nb_votes = 0
    for j in range(img.shape[0]):
        for i in range(img.shape[1]):
            if representation_score[j, i] < representation_threshold:
                continue
            binn = pow(math.e, -gradient[j,i]) * (1 - pow(math.e, -representation_score[j,i]))
            if isinstance(binn, np.ndarray):
                binn = binn[0]
            votes[j,i] = binn
            nb_votes += 1
            if binn in votes_per_bin:
                votes_per_bin[binn] += 1
            else:
                votes_per_bin[binn] = 1

    # Getting the most popular bin
    if nb_votes == 0:
        return -1, [], 0
    votes_per_bin = dict(sorted(votes_per_bin.items(), key=lambda item:item[1], reverse=True))
    winner_bin = list(votes_per_bin.items())[0][0]
    return (winner_bin, votes, nb_votes)

# Returns Pixels In Per Bin
def getPopularBinPixelCoords(votes, bin):
    # We Get All Pixels That Belong To The Bin
    coords = []
    for j in range(votes.shape[0]):
        for i in range(votes.shape[1]):
            if votes[j,i] == bin:
                coords.append((j, i))
    return coords

# Returns Seed Pixel from a bin
def getSeedPixel(img, coords, gradient, size=20):
    # Getting The Next Seed Pixel
    scores = []
    for pixel in coords:
        # 20x20 Kernel Limits
        lower_h = pixel[0]+size if pixel[1]+size < img.shape[1] else img.shape[1]-1
        higher_h = pixel[0]-size if pixel[1]-size >= 0 else 0

        lower_l = pixel[1]-size if pixel[1]-size > 0 else 0
        higher_l = pixel[1]+size if pixel[1]+size < img.shape[1] else img.shape[1]-1

        S = 0
        for j in range(lower_h, higher_h):
            for i in range(lower_l, higher_l):
                if votes[j,i] == winner_bin:
                    S += 1
        score = S*pow(math.e, -gradient[pixel])
    scores.append(score)

    # Getting The Pixel Coordinates With The Higher Score
    seed_coords = coords[np.argmax(scores)]
    return seed_coords

# Returns distribution parameters
def estimateDistribution(img, seed_coords, size=20):
    # Distribution Estimation
    lower_h = seed_coords[0]+size if seed_coords[0]+size < img.shape[1] else img.shape[1]-1
    higher_h = seed_coords[0]-size if seed_coords[0]-size >= 0 else 0

    lower_l = seed_coords[1]-size if seed_coords[1]-size > 0 else 0
    higher_l = seed_coords[1]+size if seed_coords[1]+size < img.shape[1] else img.shape[1]-1

    patch = img[higher_h:lower_h, lower_l:higher_l]
    #dist_weights = guidedFilter(patch, patch, 20, 0.09)

    mean_vector = np.mean(patch.reshape(-1, 3), axis=0)
    covariance_matrix = np.cov(patch.reshape(-1, 3), rowvar=False, bias=True)

    distribution = (mean_vector, covariance_matrix)
    return distribution

# Returns mahalanobis distance of image and distribution
def getMahalanobis(img, distribution, representation_score):
    Di = np.full((img.shape[0], img.shape[1]), 255)
    # Foreach Pixel
    for j in range(img.shape[0]):
        for i in range(img.shape[1]):
            # Inverting Covariance Matrix
            if np.linalg.cond(distribution[1]) < 1/sys.float_info.epsilon:
                inv_cov = np.linalg.inv(distribution[1])
            else:
                inv_cov = distribution[1]
            # Calculating mahalanobis distance
            Di[j,i] = np.minimum(representation_score[j,i], mahalanobis(img[j,i], distribution[0], inv_cov))
    return Di.astype(int)

#Reading Image
print('Reading Image...')
img = cv2.imread('assets/f.jpg')
# width = int(img.shape[1] * 100 / 100)
# height = int(img.shape[0] * 100 / 100)
# dim = (height, width)
#img = cv2.resize(img, dim)

#Color Model Representation
seeds = []
alpha_layers = []
color_layers = []
distributions = []
representation_scores = []
representation_threshold = 25

#Calculating image gradient
print('Calculating Gradient...')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
laplacian = cv2.Laplacian(img_gray,cv2.CV_64F)

#Initialization
print('Initializing Layers...')
alpha_layers.append(np.ones((img.shape[0], img.shape[1], 1)))
color_layers.append(np.copy(img))
representation_scores.append(np.full((img.shape[0], img.shape[1], 1), 255))

Reading Image...
Calculating Gradient...
Initializing Layers...


In [2]:
# Getting Votes + Most Popular Bin
print('Voting On Bin...')
winner_bin, votes, nb_votes = voteOnBin(img, laplacian, representation_scores[-1], representation_threshold)

# Getting Coordinates Of Pixels In The Winner Bin
print('Retreiving Pixel Coordinates...')
coords = getPopularBinPixelCoords(votes, winner_bin)

# Getting The Next Seed Pixel
print('Finding Seed Pixel...')
seed_coords = getSeedPixel(img, coords, laplacian)
seeds.append(seed_coords)

# Getting Distribution Parameters
print('Estimating Distribution...')
distribution = estimateDistribution(img, seed_coords)
distributions.append(distribution)

Voting On Bin...
Retreiving Pixel Coordinates...
Finding Seed Pixel...
Estimating Distribution...


In [7]:
def estimateRepresentation(img, distributions, representation_score):
    # Getting the minimum mahalanobis distance
    minDi = getMahalanobis(img, distributions[-1], representation_score)
    return minDi

In [8]:
representation_score = np.empty((img.shape[0], img.shape[1]))
# Multithreading
nb_threads = 4
step = math.ceil(img.shape[0]/nb_threads)
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    increment = 0
    for index in range(nb_threads):
        start = increment
        finish = start+step if (start+step) <= img.shape[0] else img.shape[0]
        future = executor.submit(estimateRepresentation, img[start:finish,:], distributions, representation_scores[-1][start:finish,:])
        representation_score[start:finish,:] = future.result()
        increment += step

In [9]:
cv2.imwrite('rep.png', representation_score)

True